In [1]:
def str_to_vec(str_word_vec):
    str_word_vec = str_word_vec[12:-33]
    str_word_vec = re.sub(r'\[', '', str_word_vec)
    str_word_vec = re.sub(r'\]', '', str_word_vec)
    str_word_vec = re.sub(r'\n', ' ', str_word_vec)
    str_word_vec = re.sub(r'\s+', ' ', str_word_vec)
    vec = np.fromstring(str_word_vec, dtype=float, sep=' ')
    return vec

def str_to_vec_rep(str_word_vec):
    str_word_vec = str_word_vec[12:-33]
    str_word_vec = re.sub(r'\[', '', str_word_vec)
    str_word_vec = re.sub(r'\]', '', str_word_vec)
    str_word_vec = re.sub(r'\n', ' ', str_word_vec)
    str_word_vec = re.sub(r'\s+', ' ', str_word_vec)
    vec = np.fromstring(str_word_vec, dtype=float, sep=' ')
    return vec

def str_to_vec_rep_old(str_word_vec):
    str_word_vec = str_word_vec[79:-25]
    str_word_vec = re.sub(r'\[', '', str_word_vec)
    str_word_vec = re.sub(r'\]', '', str_word_vec)
    str_word_vec = re.sub(r'\n', ' ', str_word_vec)
    str_word_vec = re.sub(r'\s+', ' ', str_word_vec)
    vec = np.fromstring(str_word_vec, dtype=float, sep=',')
    return vec

In [6]:
################################################## LOAD DATA ###########################################################
import pandas as pd
import numpy as np
import re
import pickle

pos = pd.read_csv('pos_yago_7.csv').sample(frac=1)
neg = pd.read_csv('neg_yago_7.csv').sample(frac=1)

# pos = pd.read_csv('pos_7.csv').sample(frac=1)
# neg = pd.read_csv('neg_7.csv').sample(frac=1)

# *********
# pos_train = pos.head(int(pos.shape[0] * 0.8))
# pos_test = pos.tail(int(pos.shape[0] * 0.2))

# neg_train = neg.head(int(neg.shape[0] * 0.8))
# neg_test = neg.tail(int(neg.shape[0] * 0.2))

# pos_train.to_csv("yago_pos_train.csv")
# pos_test.to_csv("yago_pos_test.csv")

# neg_train.to_csv("yago_neg_train.csv")
# neg_test.to_csv("yago_neg_test.csv")

# pos_train = pd.read_csv("tac_pos_train.csv")
# pos_test = pd.read_csv("tac_pos_test.csv")

# neg_train = pd.read_csv("tac_neg_train.csv")
# neg_test = pd.read_csv("tac_neg_test.csv")

# ***********

# pos_unseen = pd.read_csv('zero_shot_pov_pairs.csv')
# neg_unseen = pd.read_csv('zero_shot_neg_pairs.csv')

# pos_train = pd.read_csv('zero_shot_pos_pairs_train.csv')
# neg_train = pd.read_csv('zero_shot_neg_pairs_train.csv')


############################################### GENERATE VECTORS #######################################################
from collections import defaultdict

yago_paper_step_1_with_triv = pd.DataFrame()
yago_paper_step_1_non_triv = pd.DataFrame()

auc_lst = []
err_lst = []
precision_lst = []
recall_last = []
f1_lst = []

agg_metrics_with_triv = defaultdict(list)
agg_metrics_non_triv = defaultdict(list)

metrics_with_triv = defaultdict(list)
metrics_non_triv = defaultdict(list)

for prct in [0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]:
    with_trivial = defaultdict(list)
    non_trivial = defaultdict(list)
    for i in range(1, 11):
        final_vecs_pos = []
        final_vecs_neg = []
        pos_labels = []
        neg_labels = []
        
        final_vecs_pos_test = []
        final_vecs_neg_test = []
        pos_labels_test = []
        neg_labels_test = []
        y = 0
        # pos_labels = np.array(pos['same_entity'])
        # neg_labels = np.array(neg['same_entity'])
#         pos = pos.sample(frac=1)
#         neg = neg.sample(frac=1)

        for index, row in pos.iterrows():
        #     try:
                diff_vec = []
                a_word_vec = str_to_vec_rep(row['a_word_vec'])
                b_word_vec = str_to_vec_rep(row['b_word_vec'])

                a_sentence_vec = str_to_vec_rep(row['a_sen_vec'])
                b_sentence_vec = str_to_vec_rep(row['b_sen_vec'])

                a_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_a'])
                b_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_b'])

                diff_vec.append(np.concatenate([(b_word_vec - a_word_vec) ** 2, 
                                                (b_sentence_vec - a_sentence_vec) ** 2,
                                                (b_sentence_rep_vec - a_sentence_rep_vec) ** 2,
                                                (a_sentence_rep_vec - a_sentence_vec) ** 2,
                                                (b_sentence_rep_vec - b_sentence_vec) ** 2]))

                diff_vec.append(row['edit_distance'])
                diff_vec.append(row['capitalized_edit_distance'])
                diff_vec.append(row['mention_b'])
                diff_vec.append(row['mention_a'])
                diff_vec.append(row['text_b'])
                diff_vec.append(row['text_a'])
                final_vecs_pos.append(diff_vec)
                pos_labels.append(1)
        #     except:
        #         print(index)
        #         print(b_sentence_rep_vec)
        #         print('*************', len(b_sentence_rep_vec), '**************')
        # final_vecs_pos = np.array(final_vecs_pos)

        for index, row in neg.iterrows():
        #     try:
                diff_vec = []
                a_word_vec = str_to_vec_rep(row['a_word_vec'])
                b_word_vec = str_to_vec_rep(row['b_word_vec'])

                a_sentence_vec = str_to_vec_rep(row['a_sen_vec'])
                b_sentence_vec = str_to_vec_rep(row['b_sen_vec'])

                a_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_a'])
                b_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_b'])

                diff_vec.append(np.concatenate([(b_word_vec - a_word_vec) ** 2, 
                                                (b_sentence_vec - a_sentence_vec) ** 2,
                                                (b_sentence_rep_vec - a_sentence_rep_vec) ** 2,
                                                (a_sentence_rep_vec - a_sentence_vec) ** 2,
                                                (b_sentence_rep_vec - b_sentence_vec) ** 2]))
                diff_vec.append(row['edit_distance'])
                diff_vec.append(row['capitalized_edit_distance'])
                diff_vec.append(row['mention_b'])
                diff_vec.append(row['mention_a'])
                diff_vec.append(row['text_b'])
                diff_vec.append(row['text_a'])
                final_vecs_neg.append(diff_vec)
                neg_labels.append(0)
        #     except:
        #         print(index)
        #         print('index_b: ', index)
        # final_vecs_neg = np.array(final_vecs_neg)
        
        
#         for index, row in pos_test.iterrows():
#         #     try:
#                 diff_vec = []
#                 a_word_vec = str_to_vec_rep(row['a_word_vec'])
#                 b_word_vec = str_to_vec_rep(row['b_word_vec'])

#                 a_sentence_vec = str_to_vec_rep(row['a_sen_vec'])
#                 b_sentence_vec = str_to_vec_rep(row['b_sen_vec'])

#                 a_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_a'])
#                 b_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_b'])

#                 diff_vec.append(np.concatenate([(b_word_vec - a_word_vec) ** 2, 
#                                                 (b_sentence_vec - a_sentence_vec) ** 2,
#                                                 (b_sentence_rep_vec - a_sentence_rep_vec) ** 2,
#                                                 (a_sentence_rep_vec - a_sentence_vec) ** 2,
#                                                 (b_sentence_rep_vec - b_sentence_vec) ** 2]))

#                 diff_vec.append(row['edit_distance'])
#                 diff_vec.append(row['capitalized_edit_distance'])
#                 diff_vec.append(row['mention_b'])
#                 diff_vec.append(row['mention_a'])
#                 diff_vec.append(row['text_b'])
#                 diff_vec.append(row['text_a'])
#                 final_vecs_pos_test.append(diff_vec)
#                 pos_labels_test.append(1)
#         #     except:
#         #         print(index)
#         #         print(b_sentence_rep_vec)
#         #         print('*************', len(b_sentence_rep_vec), '**************')
#         # final_vecs_pos = np.array(final_vecs_pos)

#         for index, row in neg_test.iterrows():
#         #     try:
#                 diff_vec = []
#                 a_word_vec = str_to_vec_rep(row['a_word_vec'])
#                 b_word_vec = str_to_vec_rep(row['b_word_vec'])

#                 a_sentence_vec = str_to_vec_rep(row['a_sen_vec'])
#                 b_sentence_vec = str_to_vec_rep(row['b_sen_vec'])

#                 a_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_a'])
#                 b_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_b'])

#                 diff_vec.append(np.concatenate([(b_word_vec - a_word_vec) ** 2, 
#                                                 (b_sentence_vec - a_sentence_vec) ** 2,
#                                                 (b_sentence_rep_vec - a_sentence_rep_vec) ** 2,
#                                                 (a_sentence_rep_vec - a_sentence_vec) ** 2,
#                                                 (b_sentence_rep_vec - b_sentence_vec) ** 2]))
#                 diff_vec.append(row['edit_distance'])
#                 diff_vec.append(row['capitalized_edit_distance'])
#                 diff_vec.append(row['mention_b'])
#                 diff_vec.append(row['mention_a'])
#                 diff_vec.append(row['text_b'])
#                 diff_vec.append(row['text_a'])
#                 final_vecs_neg_test.append(diff_vec)
#                 neg_labels_test.append(0)
#         #     except:
#         #         print(index)
#         #         print('index_b: ', index)
#         # final_vecs_neg = np.array(final_vecs_neg)



        ############################################### VERIFY VECTORS #########################################################


        res_pos = []
        pos_l = []
        for v in final_vecs_pos:
            if len(v[0]) == 2560:
                res_pos.append(v)
                pos_l.append(1)

        res_neg = []
        neg_l = []
        for v in final_vecs_neg:
            if len(v[0]) == 2560:
                res_neg.append(v)
                neg_l.append(0)

        res_pos = np.array(res_pos)
        res_neg = np.array(res_neg)
        pos_l = np.array(pos_l)
        neg_l = np.array(neg_l)
        
#         res_pos_test = []
#         pos_l_test = []
#         for v in final_vecs_pos_test:
#             if len(v[0]) == 2560:
#                 res_pos_test.append(v)
#                 pos_l_test.append(1)

#         res_neg_test = []
#         neg_l_test = []
#         for v in final_vecs_neg_test:
#             if len(v[0]) == 2560:
#                 res_neg_test.append(v)
#                 neg_l_test.append(0)

#         res_pos_test = np.array(res_pos_test)
#         res_neg_test = np.array(res_neg_test)
#         pos_l_test = np.array(pos_l_test)
#         neg_l_test = np.array(neg_l_test)


        ############################################# TRAIN TEST SPLIT #########################################################


        from sklearn.model_selection import train_test_split

        train_features_pos, test_features_pos, train_labels_pos, test_labels_pos = train_test_split(res_pos, pos_l, test_size = prct)
    #     print('POSITIVE')
    #     print('train features: ', train_features_pos.shape)
    #     print('test features: ', test_features_pos.shape)
    #     print('train labels: ', train_labels_pos.shape)
    #     print('test labels: ', test_labels_pos.shape)
    #     print('\n')

        train_features_neg, test_features_neg, train_labels_neg, test_labels_neg = train_test_split(res_neg, neg_l, test_size = prct)
    #     print('NEGATIVE')
    #     print('train features: ', train_features_neg.shape)
    #     print('test features: ', test_features_neg.shape)
    #     print('train labels: ', train_labels_neg.shape)
    #     print('test labels: ', test_labels_neg.shape)
    #     print('\n')

        train_features_pos = list(train_features_pos)
        train_features_pos.extend(list(train_features_neg))
        train_features_list = train_features_pos
        train_features = [x[0] for x in train_features_pos]
        train_features = np.array(train_features_pos)
        
        test_features_pos = list(test_features_pos)
        test_features_pos.extend(list(test_features_neg))
        test_features_list = test_features_pos
        test_features = [x[0] for x in test_features_pos]
        test_features = np.array(test_features_pos)

#         test_features_pos = list(res_pos_test)
#         test_features_pos.extend(list(res_neg_test))
#         test_features_list = test_features_pos
#         test_features = [x[0] for x in test_features_pos]
#         test_features = np.array(test_features_pos)

        train_labels_pos = list(train_labels_pos)
        train_labels_pos.extend(list(train_labels_neg))
        train_labels = np.array(train_labels_pos)

        test_labels_pos = list(test_labels_pos)
        test_labels_pos.extend(list(test_labels_neg))
        test_labels = np.array(test_labels_pos)

    #     print('TOTAL')
    #     print(train_features.shape)
    #     print(test_features.shape)
    #     print(train_labels.shape)
    #     print(test_labels.shape)
    #     print('\n')


        ############################################# ALL DATA RESULTS  ########################################################


        from sklearn.ensemble import RandomForestClassifier
        from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
        import pandas as pd
        import numpy as np
        
        train_same_men = np.mean([t[0][:512] for t in train_features])
        test_same_men = np.mean([t[0][:512] for t in test_features])
        
        rf = RandomForestClassifier(n_estimators = 100)
        train_set = [np.append(np.append(t[0], t[1]), t[2]) for t in train_features]
        rf.fit(train_set, train_labels)
        
        with open('rf_yago.pickle', 'wb') as handle:
            pickle.dump(rf, handle)

        predictions = rf.predict([np.append(np.append(t[0], t[1]), t[2]) for t in test_features])
        predictions_probs = rf.predict_proba([np.append(np.append(t[0], t[1]), t[2]) for t in test_features])
        errors = abs(predictions - test_labels)
        roc_auc = roc_auc_score(test_labels, predictions)

        print('All Results')
        print('roc auc: ', roc_auc)
        print('error: ', sum(errors)/len(predictions))
        print('f1: ', f1_score(test_labels, predictions))
        print('precision :', precision_score(test_labels, predictions))
        print('recall :', recall_score(test_labels, predictions))  
        print('***********************')
        confusion_df = pd.DataFrame(confusion_matrix(test_labels, predictions))
        confusion_df.columns = ['predicted False', 'predicted True']
        confusion_df[''] = ['actual False', 'actual True']
        print(confusion_df.to_string(index=False))
        print('\n\n')

        with_trivial['auc'].append(roc_auc)
        with_trivial['error'].append(sum(errors)/len(predictions))
        with_trivial['precision'].append(precision_score(test_labels, predictions))
        with_trivial['recall'].append(recall_score(test_labels, predictions))
        with_trivial['f1'].append(f1_score(test_labels, predictions))
        with_trivial["train_same_men"].append(train_same_men)
        with_trivial["test_same_men"].append(test_same_men)

        ############################################# NON TRIVIAL RESULTS  ########################################################


        from sklearn.ensemble import RandomForestClassifier
        from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
        import pandas as pd
        
        test_features_not_trivial = []
        test_labels_not_trivial = []
        for i, t in enumerate(test_features):
            if (t[3] == t[4] and test_labels[i] == 0) or (t[3] != t[4] and test_labels[i] == 1):
                test_features_not_trivial.append(t)
                test_labels_not_trivial.append(test_labels[i])
        
        train_same_men = np.mean([t[0][:512] for t in train_features])
        test_same_men = np.mean([t[0][:512] for t in test_features_not_trivial])
        
        predictions = rf.predict([np.append(np.append(t[0], t[1]), t[2]) for t in test_features_not_trivial])
        predictions_probs = rf.predict_proba([np.append(np.append(t[0], t[1]), t[2]) for t in test_features_not_trivial])
        errors = abs(predictions - test_labels_not_trivial)
        roc_auc = roc_auc_score(test_labels_not_trivial, predictions)

        print('Non Trivial Results')
        print('roc auc: ', roc_auc)
        print('error: ', sum(errors)/len(predictions))
        print('f1: ', f1_score(test_labels_not_trivial, predictions))
        print('precision :', precision_score(test_labels_not_trivial, predictions))
        print('recall :', recall_score(test_labels_not_trivial, predictions))  
        print('***********************')
        confusion_df = pd.DataFrame(confusion_matrix(test_labels_not_trivial, predictions))
        confusion_df.columns = ['predicted False', 'predicted True']
        confusion_df[''] = ['actual False', 'actual True']
        print(confusion_df.to_string(index=False))

        non_trivial['auc'].append(roc_auc)
        non_trivial['error'].append(sum(errors)/len(predictions))
        non_trivial['precision'].append(precision_score(test_labels_not_trivial, predictions))
        non_trivial['recall'].append(recall_score(test_labels_not_trivial, predictions))
        non_trivial['f1'].append(f1_score(test_labels_not_trivial, predictions))
        non_trivial["train_same_men"].append(train_same_men)
        non_trivial["test_same_men"].append(test_same_men)
        
        print("#################################################")
    ############################################# AGGREGATE RESULTS ################################################################


    import statistics
    
    print(f'# Positive Pairs: {prct}')

    for key in with_trivial.keys():
        agg_metrics_with_triv[key].append(statistics.mean(with_trivial[key]))
#         metrics_with_triv.append(with_trivial[key])

    for key in non_trivial.keys():
        agg_metrics_non_triv[key].append(statistics.mean(non_trivial[key]))
#         metrics_non_triv.append(non_trivial[key])
                
#     print(agg_metrics_with_triv)
#     print(agg_metrics_non_triv)
############################################# SAVE CSVs ################################################################


yago_paper_step_1_with_triv['#_pos_pairs'] = [0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
yago_paper_step_1_with_triv['auc'] = agg_metrics_with_triv['auc']
yago_paper_step_1_with_triv['error'] = agg_metrics_with_triv['error']
yago_paper_step_1_with_triv['precision'] = agg_metrics_with_triv['precision']
yago_paper_step_1_with_triv['recall'] = agg_metrics_with_triv['recall']
yago_paper_step_1_with_triv['f1'] = agg_metrics_with_triv['f1']
yago_paper_step_1_with_triv.to_csv('last_2_yago_paper_step_1_with_triv.csv')

yago_paper_step_1_non_triv['#_pos_pairs'] = [0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
yago_paper_step_1_non_triv['auc'] = agg_metrics_non_triv['auc']
yago_paper_step_1_non_triv['error'] = agg_metrics_non_triv['error']
yago_paper_step_1_non_triv['precision'] = agg_metrics_non_triv['precision']
yago_paper_step_1_non_triv['recall'] = agg_metrics_non_triv['recall']
yago_paper_step_1_non_triv['f1'] = agg_metrics_non_triv['f1']
yago_paper_step_1_non_triv.to_csv('last_2_yago_paper_step_1_non_triv.csv')

<ipython-input-1-8f90cbef799c>:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vec = np.fromstring(str_word_vec, dtype=float, sep=' ')


All Results
roc auc:  0.6101671256557204
error:  0.38990308767185033
f1:  0.6026642168121268
precision : 0.6165413533834586
recall : 0.5893980233602875
***********************
 predicted False  predicted True              
            2790            1632  actual False
            1828            2624   actual True



Non Trivial Results
roc auc:  0.2715795486423086
error:  0.7283588818755635
f1:  0.2808813710215891
precision : 0.27883340698188247
recall : 0.28295964125560535
***********************
 predicted False  predicted True              
             574            1632  actual False
            1599             631   actual True
#################################################


<ipython-input-1-8f90cbef799c>:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vec = np.fromstring(str_word_vec, dtype=float, sep=' ')


All Results
roc auc:  0.5985211165442087
error:  0.4013973405454136
f1:  0.6088293432901383
precision : 0.5956166738289643
recall : 0.6226415094339622
***********************
 predicted False  predicted True              
            2540            1882  actual False
            1680            2772   actual True



Non Trivial Results
roc auc:  0.20752649012012112
error:  0.7922868741542625
f1:  0.25239412640987446
precision : 0.23959595959595958
recall : 0.266636690647482
***********************
 predicted False  predicted True              
             328            1882  actual False
            1631             593   actual True
#################################################


<ipython-input-1-8f90cbef799c>:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vec = np.fromstring(str_word_vec, dtype=float, sep=' ')


...

All Results
roc auc:  0.8527442207307979
error:  0.14715719063545152
f1:  0.857451403887689
precision : 0.8340336134453782
recall : 0.8822222222222222
***********************
 predicted False  predicted True              
             368              79  actual False
              53             397   actual True



Non Trivial Results
roc auc:  0.7606239326422812
error:  0.23608017817371937
f1:  0.792156862745098
precision : 0.7240143369175627
recall : 0.8744588744588745
***********************
 predicted False  predicted True              
             141              77  actual False
              29             202   actual True
#################################################


<ipython-input-1-8f90cbef799c>:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vec = np.fromstring(str_word_vec, dtype=float, sep=' ')


All Results
roc auc:  0.8739224459358688
error:  0.12597547380156077
f1:  0.8781014023732471
precision : 0.8532494758909853
recall : 0.9044444444444445
***********************
 predicted False  predicted True              
             377              70  actual False
              43             407   actual True



Non Trivial Results
roc auc:  0.7734489915170565
error:  0.2219730941704036
f1:  0.8047337278106508
precision : 0.7445255474452555
recall : 0.8755364806866953
***********************
 predicted False  predicted True              
             143              70  actual False
              29             204   actual True
#################################################


<ipython-input-1-8f90cbef799c>:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vec = np.fromstring(str_word_vec, dtype=float, sep=' ')


All Results
roc auc:  0.8571066368381806
error:  0.1426978818283166
f1:  0.865546218487395
precision : 0.8207171314741036
recall : 0.9155555555555556
***********************
 predicted False  predicted True              
             357              90  actual False
              38             412   actual True



Non Trivial Results
roc auc:  0.7547102483585497
error:  0.24830699774266365
f1:  0.7808764940239044
precision : 0.6877192982456141
recall : 0.9032258064516129
***********************
 predicted False  predicted True              
             137              89  actual False
              21             196   actual True
#################################################
# Positive Pairs: 0.1


In [74]:
not_baseline_auc_lst = [x * 100 for x in non_trivial['auc']]

In [75]:
not_baseline_auc_lst

[99.43820224719101,
 98.02386192517085,
 97.91150237460906,
 98.36094057685625,
 98.65168539325843,
 98.47330012741806,
 96.90026641955289,
 98.53932584269663,
 97.39603845708328,
 97.05895980539788]

In [76]:
baseline_auc_lst = [0.42134831460674155,
 0.8887640449438202,
 0.8786516853932584,
 0.8876404494382022,
 0.8820224719101124,
 0.8786516853932584,
 0.8853932584269664,
 0.8943820224719101,
 0.43258426966292135,
 0.8831460674157303]

In [77]:
baseline_auc_lst = [x * 100 for x in baseline_auc_lst]

In [78]:
from scipy.stats import chisquare, chi2_contingency

chisquare(f_exp=not_baseline_auc_lst, f_obs=baseline_auc_lst)

Power_divergenceResult(statistic=70.53488990041409, pvalue=1.1958888575260835e-11)

In [71]:
chisquare(f_exp=[0.69] * 10, f_obs=[0.75] * 10)

Power_divergenceResult(statistic=0.052173913043478355, pvalue=0.9999999986011135)